# Compare direct solve vs Jacobi-GMRES in bw2calc

This notebook runs the same LCA twice:

- `LCA` (direct sparse solve)
- `JacobiGMRESLCA` (Jacobi-preconditioned GMRES with fallback)

It then compares runtime and numerical agreement.

In [1]:
import numpy as np

from bw2calc import LCA, JacobiGMRESLCA
import bw2data

In [2]:
bw2data.projects.set_current("ecoinvent-3.10-cutoff")

In [3]:
method = [m for m in bw2data.methods if "GWP100" in str(m)][0]
method

('CML v4.8 2016 no LT',
 'climate change no LT',
 'global warming potential (GWP100) no LT')

In [4]:
db = bw2data.Database("ecoinvent-3.10.1-cutoff")

In [5]:
act = [a for a in db if a["name"].startswith("passenger car production, diesel") and a["location"] == "GLO"][0]
act

'passenger car production, diesel' (kilogram, GLO, None)

### Attempt with regular-size database

The wall time used to be 4.2 seconds before using pandas.unique() in matrix_utils.

In [8]:
%%time

lca = LCA({act:1}, method)
lca.lci()
lca.lcia()
print(lca.score)

7.411258715859462
CPU times: user 652 ms, sys: 73.7 ms, total: 726 ms
Wall time: 544 ms


In [9]:
%%time

lca = JacobiGMRESLCA({act:1}, method)
lca.lci()
lca.lcia()
print(lca.score)

7.411258715477395
CPU times: user 284 ms, sys: 114 ms, total: 398 ms
Wall time: 350 ms


### Attempt with large-size database

In [4]:
db = bw2data.Database("ecoinvent-3.10.1-cutoff - regionalized")
len(db)

251593

In [5]:
act = [a for a in db if a["name"].startswith("passenger car production, diesel") and a["location"] == "GLO"][0]
act

'passenger car production, diesel' (kilogram, GLO, None)

The wall time used to be 15 minutes before using pandas.unique() in matrix_utils.

In [13]:
%%time

lca = LCA({act:1}, method)
lca.lci()
lca.lcia()
print(lca.score)

6.049230549617069
CPU times: user 17min 58s, sys: 13min 31s, total: 31min 29s
Wall time: 10min 34s


/opt/homebrew/Caskroom/miniforge/base/envs/bw2calc/lib/python3.12/site-packages/scikits/umfpack/umfpack.py:737: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.69e+14)
  warnings.warn(msg, UmfpackWarning)


In [6]:
%%time

lca = JacobiGMRESLCA({act:1}, method)
lca.lci()
lca.lcia()
print(lca.score)

6.049230545497951
CPU times: user 3.65 s, sys: 588 ms, total: 4.24 s
Wall time: 4.01 s


In [7]:
import numpy as np

In [9]:
np.array([17.4357463 , 11.03358014, 17.09238729, 13.20164585, 15.03311766, 4.20133189, 15.45619411, 14.72765565, 16.20026822,  7.07807607]).mean()

np.float64(13.146000318)